In [2]:
import numpy as np

# ДЗ1: Puma с кистью

<img src="images/schema.jpg" alt="Кинематическая схема">
<p align="center"><em>Кинематическая схема</em></p>

Вспомогательные функции

In [3]:
def Rx(q):
    T = np.array([[1,         0,          0, 0],
                  [0, np.cos(q), -np.sin(q), 0],
                  [0, np.sin(q),  np.cos(q), 0],
                  [0,         0,          0, 1]], dtype=float)
    return T


def Ry(q):
    T = np.array([[ np.cos(q), 0, np.sin(q), 0],
                  [         0, 1,         0, 0],
                  [-np.sin(q), 0, np.cos(q), 0],
                  [         0, 0,         0, 1]], dtype=float)
    return T


def Rz(q):
    T = np.array([[np.cos(q), -np.sin(q), 0, 0],
                  [np.sin(q),  np.cos(q), 0, 0],
                  [        0,          0, 1, 0],
                  [        0,          0, 0, 1]], dtype=float)
    return T


def Tx(x):
    T = np.array([[1, 0, 0, x],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=float)
    return T


def Ty(y):
    T = np.array([[1, 0, 0, 0],
                  [0, 1, 0, y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=float)
    return T


def Tz(z):
    T = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, z],
                  [0, 0, 0, 1]], dtype=float)
    return T

def T_next(theta_i, d_i, alpha_i, a_i):
    T_i = Rz(theta_i) @ Tz(d_i) @ Tx(a_i) @ Rx(alpha_i)
    return T_i

## Кинематическая схема

## Решение прямой задачи

<img src="images/dh.jpg" alt="Денавита-Хартенберга">
<p align="center"><em>Решение с помощью метода Денавита–Хартенберга</em></p>

Зададим длины

In [4]:
l1 = 3
l2 = 2
l3 = 2

Задаем углы, которые достаточно понятны для понимания (вытянутый 2 шарнир вправо)

In [5]:
theta0 = 0
theta1 = 0
theta2 = 90
theta3 = 0
theta4 = 180
theta5 = 10

theta0_rad = np.deg2rad(theta0)
theta1_rad = np.deg2rad(theta1)
theta2_rad = np.deg2rad(theta2)
theta3_rad = np.deg2rad(theta3)
theta4_rad = np.deg2rad(theta4)
theta5_rad = np.deg2rad(theta5)

T = T_next(theta0_rad, l1, np.pi/2, 0) @ T_next(theta1_rad, 0, 0, l2) @ T_next(theta2_rad, 0, np.pi/2, 0) @ T_next(-theta3_rad, l3, -np.pi/2, 0) @ T_next(-theta4_rad, 0, np.pi/2, 0) @ T_next(theta5_rad, 0, 0, 0)
print(T)

[[ 7.09349674e-17  4.96691989e-17 -1.00000000e+00  4.00000000e+00]
 [-1.73648178e-01 -9.84807753e-01 -6.12323400e-17 -1.22464680e-16]
 [-9.84807753e-01  1.73648178e-01 -6.12323400e-17  3.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Решение обратной задачи

Обратную кинематику решал на основе предположений:

* `z = l1 + l2 * sin(Θ1) - l3 * cos(Θ1 + Θ2)`
* `x = (l2 * cos(Θ1) + l3 * sin(Θ1 + Θ2)) * cos(Θ0)`
* `y = (l2 * cos(Θ1) + l3 * sin(Θ1 + Θ2)) * sin(Θ0)`

Координаты не зависят от сферического запястья, его можно не учитывать на позиционном этапе. Далее углы запястья ищутся по стандартным формулам для сферического шарнира Z–Y–Z: зная итоговую `T`, находим `T` до запястья и берём оставшуюся ориентацию `R`, из которой извлекаем углы запястья.

In [6]:
def euler_zyz_from_R(R, degrees=False, z_down_y_toward=True, eps=1e-12):
    '''Функция для нахождения углов в сферическо шарнире zyz (учитывает что мои оси отрицательно направлены относительно глобальной системы)'''
    R = np.asarray(R, dtype=float)
    if R.shape != (3, 3):
        raise ValueError("R must be 3x3")

    if z_down_y_toward:
        S = np.diag([1.0, -1.0, -1.0])
        R = S @ R @ S

    r11, r12, r13 = R[0]
    r21, r22, r23 = R[1]
    r31, r32, r33 = R[2]

    beta = np.arctan2(np.hypot(r13, r23), r33)

    if abs(np.sin(beta)) > eps:
        alpha = np.arctan2(r23, r13)
        gamma = np.arctan2(r32, -r31)
    else:
        if r33 > 0:
            alpha = 0.0
            gamma = np.arctan2(r21, r11)
        else:
            alpha = 0.0
            gamma = np.arctan2(-r21, -r11)

    def wrap_pi(a):
        return (a + np.pi) % (2*np.pi) - np.pi
    alpha, beta, gamma = map(wrap_pi, (alpha, beta, gamma))

    if degrees:
        alpha, beta, gamma = map(np.degrees, (alpha, beta, gamma))

    return alpha, beta, gamma

def inverse_puma(T, degrees=True, eps=1e-12):
    x, y, z = float(T[0, -1]), float(T[1, -1]), float(T[2, -1])

    theta0 = np.arctan2(y, x)

    A = np.sqrt(x**2 + y**2)
    B = z - l1

    denom = 2.0 * l2 * l3
    if abs(denom) < eps:
        raise ValueError("Дегенератная геометрия: l2*l3≈0.")

    sin_theta2 = (A*A + B*B - l2*l2 - l3*l3) / denom
    sin_theta2 = float(np.clip(sin_theta2, -1.0, 1.0))

    cos_sq = max(0.0, 1.0 - sin_theta2**2)
    cos_pos = np.sqrt(cos_sq)
    cos_neg = -cos_pos

    sols = []
    R06 = np.asarray(T[:3, :3], float)

    for cos_t2 in (cos_pos, cos_neg):
        R = l2 + l3 * sin_theta2
        S = l3 * cos_t2
        denom_RS = R*R + S*S
        if denom_RS < eps:
            continue

        sin_t1 = (S*A + R*B) / denom_RS
        cos_t1 = (R*A - S*B) / denom_RS
        sin_t1 = float(np.clip(sin_t1, -1.0, 1.0))
        cos_t1 = float(np.clip(cos_t1, -1.0, 1.0))

        theta1 = np.arctan2(sin_t1, cos_t1)
        theta2 = np.arctan2(sin_theta2, cos_t2)

        R03 = Ry(theta2) @ Ry(theta1) @ Rz(theta0)
        R36 = R03[:3, :3].T @ R06

        alpha, beta, gamma = euler_zyz_from_R(R36, degrees=False, z_down_y_toward=True, eps=eps)

        theta3 = -alpha
        theta4 = -beta
        theta5 =  gamma

        def wrap_pi(a):
          return (a + np.pi) % (2*np.pi) - np.pi

        if theta4 < 0:
            theta3 -= np.pi
            theta4 = -theta4
            theta5 += np.pi

        if theta5 < 0:
            theta3 -= np.pi
            theta4 = -theta4
            theta5 += np.pi

        theta3, theta4, theta5 = map(wrap_pi, (theta3, theta4, theta5))

        if degrees:
            sols.append((
                np.degrees(theta0).item(),
                np.degrees(theta1).item(),
                np.degrees(theta2).item(),
                np.degrees(theta3).item(),
                np.degrees(theta4).item(),
                np.degrees(theta5).item(),
            ))
        else:
            sols.append((theta0, theta1, theta2, theta3, theta4, theta5))

    return sols

In [7]:
T = np.array([
    [0, -1, 0, 4],
    [-1, 0, 0, 0],
    [0, 0, -1, 3],
    [0, 0, 0, 1]
])
sol = inverse_puma(T)
sol

[(0.0, 0.0, 90.0, 0.0, 90.0, 90.0), (0.0, 0.0, 90.0, 0.0, 90.0, 90.0)]

In [8]:
T = np.array([
    [1, 0, 0, 2],
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 1]
])
sol = inverse_puma(T)
sol

[(0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, -90.0, 180.0, -180.0, 90.0, -180.0)]

In [9]:
T = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 2],
    [0, 0, 1, 1],
    [0, 0, 0, 1]
])
sol = inverse_puma(T)
sol

[(90.0, 0.0, 0.0, 0.0, 0.0, 90.0), (90.0, -90.0, 180.0, 90.0, 90.0, -180.0)]

In [10]:
T = np.array([[ 7.09349674e-17, 4.96691989e-17, -1.00000000e+00, 4.00000000e+00],
 [-1.73648178e-01, -9.84807753e-01, -6.12323400e-17, -1.22464680e-16],
 [-9.84807753e-01,  1.73648178e-01, -6.12323400e-17,  3.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
sol = inverse_puma(T)
sol

[(-1.7541773258550455e-15, 0.0, 90.0, -180.0, -180.0, 10.000000018915028),
 (-1.7541773258550455e-15, 0.0, 90.0, -180.0, -180.0, 10.000000018915028)]

## Проверка

Функция генерирует углы, решает задачу прямой кинематике, находя T, а затем, используя функцию обратной кинематики, находит конфигурации. Эти конфигурации преобразуются в матрицу преобразования и сравнивают с искомой матрицей.

In [11]:
def test_fk_ik_10(tol_pos=1e-7, tol_ang=1e-7, seed=0):
    "Сгенерировать 100 случайных конфигураций, проверить FK→IK→FK на эквивалентность."
    rng = np.random.default_rng(seed)

    def wrap_pi(a): return (a + np.pi) % (2*np.pi) - np.pi
    def rot3_of(T4): return np.asarray(T4[:3, :3], float)
    def rot_err(Ra, Rb):
        tr = np.clip((np.trace(Ra.T @ Rb) - 1.0) / 2.0, -1.0, 1.0)
        return float(np.arccos(tr))
    def same_pose(Ta, Tb):
        pa, pb = np.asarray(Ta[:3, 3], float), np.asarray(Tb[:3, 3], float)
        return np.linalg.norm(pa - pb) <= tol_pos and rot_err(rot3_of(Ta), rot3_of(Tb)) <= tol_ang
    def fk(t):
        t0, t1, t2, t3, t4, t5 = t
        return T_next(theta0_rad, l1, np.pi/2, 0) @ T_next(theta1_rad, 0, 0, l2) @ T_next(theta2_rad, 0, np.pi/2, 0) @ T_next(-theta3_rad, l3, -np.pi/2, 0) @ T_next(-theta4_rad, 0, np.pi/2, 0) @ T_next(theta5_rad, 0, 0, 0)

    for _ in range(100):
        t = np.array([
            rng.uniform(-np.pi, np.pi),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi, np.pi),
            rng.uniform(0.0, np.pi),
            rng.uniform(-np.pi, np.pi),
        ], float)
        T_ref = fk(t)
        sols = inverse_puma(T_ref, degrees=False)
        if not sols:
            raise AssertionError("ИК не нашла решений")
        if not any(same_pose(T_ref, fk(s)) for s in sols):
            raise AssertionError("Ни одно решение ИК не воспроизводит FK-позу")
    return True


In [12]:
test_fk_ik_10()

True

## Визуализация

Визуализируем ошибку решения и несколько конфигураций робота

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt

l1, l2, l3 = 1.0, 4.0, 1.0

# ---- утилиты ----
def rot3_of(T4):
    return np.asarray(T4[:3, :3], float)

def same_pose(Ta, Tb, tol_pos=1e-7, tol_ang=1e-7):
    Ra, Rb = rot3_of(Ta), rot3_of(Tb)
    pa, pb = np.asarray(Ta[:3,3], float), np.asarray(Tb[:3,3], float)
    pos_ok = np.linalg.norm(pa - pb) <= tol_pos
    tr = np.clip((np.trace(Ra.T @ Rb) - 1.0)/2.0, -1.0, 1.0)
    ang_ok = float(np.arccos(tr)) <= tol_ang
    return pos_ok and ang_ok

def fk_puma(t, l1, l2, l3):
    t0, t1, t2, t3, t4, t5 = t
    return (
        T_next(t0,  l1,  np.pi/2, 0) @
        T_next(t1,  0.0, 0.0,     l2) @
        T_next(t2,  0.0, np.pi/2, 0) @
        T_next(-t3, l3, -np.pi/2, 0) @
        T_next(-t4, 0.0, np.pi/2, 0) @
        T_next(t5,  0.0, 0.0,     0)
    )

def fk_with_origins(t, l1, l2, l3):
    T = np.eye(4)
    origins = [T[:3,3].copy()]
    for A in (
        T_next(t[0],  l1,  np.pi/2, 0),
        T_next(t[1],  0.0, 0.0,     l2),
        T_next(t[2],  0.0, np.pi/2, 0),
        T_next(-t[3], l3, -np.pi/2, 0),
        T_next(-t[4], 0.0, np.pi/2, 0),
        T_next(t[5],  0.0, 0.0,     0),
    ):
        T = T @ A
        origins.append(T[:3,3].copy())
    return T, np.stack(origins, axis=0)  # финальная поза и точки суставов

def plot_arm(origins, title, filepath):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    xs, ys, zs = origins[:,0], origins[:,1], origins[:,2]
    ax.plot(xs, ys, zs, marker='o')
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
    ax.set_title(title)
    mins, maxs = origins.min(0), origins.max(0)
    center = (mins + maxs)/2
    span = float(max(maxs - mins)) or 1.0
    pad = 0.25*span
    ax.set_xlim(center[0]-span/2-pad, center[0]+span/2+pad)
    ax.set_ylim(center[1]-span/2-pad, center[1]+span/2+pad)
    ax.set_zlim(center[2]-span/2-pad, center[2]+span/2+pad)
    fig.tight_layout()
    fig.savefig(filepath, dpi=160)
    plt.close(fig)

# ---- 1) рабочее пространство (только точки, без ошибок ориентации) ----
def sample_workspace(n=2000, seed=42):
    rng = np.random.default_rng(seed)
    pts = []
    for _ in range(n):
        t = np.array([
            rng.uniform(-np.pi, np.pi),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi, np.pi),
            rng.uniform(0.0, np.pi),
            rng.uniform(-np.pi, np.pi),
        ], dtype=float)
        T = fk_puma(t, l1, l2, l3)
        pts.append(T[:3,3])
    pts = np.array(pts)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=3)
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
    ax.set_title('Рабочее пространство (облако точек)')
    fig.tight_layout()
    fig.savefig("images/workspace_points.png", dpi=160)
    plt.close(fig)

# ---- 2) примеры: одна целевая поза → все IK-ветви → картинки робота ----
def visualize_examples(num_targets=3, seed=0, tol_pos=1e-6, tol_ang=1e-6, max_per_target=12):
    rng = np.random.default_rng(seed)
    saved = []

    def to_radians_if_needed(s):
        s = np.asarray(s, float).ravel()
        if s.size != 6:
            return None
        if np.max(np.abs(s)) > 2*np.pi + 1e-3:
            s = np.deg2rad(s)
        return s

    for i in range(1, num_targets+1):
        t_true = np.array([
            rng.uniform(-np.pi, np.pi),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi/2, np.pi/2),
            rng.uniform(-np.pi, np.pi),
            rng.uniform(0.0, np.pi),
            rng.uniform(-np.pi, np.pi),
        ], dtype=float)

        Tref, _ = fk_with_origins(t_true, l1, l2, l3)

        # всегда сохраняем референсную конфигурацию
        _, origins_ref = fk_with_origins(t_true, l1, l2, l3)
        ref_path = f"images/example_{i}_config_ref.png"
        plot_arm(origins_ref, f"Цель {i}, референс\nθ[рад]={np.round(t_true,3)}", ref_path)
        saved.append(ref_path)

        try:
            sols = inverse_puma(Tref, l1=l1, l2=l2, l3=l3, degrees=False)
        except TypeError:
            sols = inverse_puma(Tref, degrees=False)

        if not sols:
            continue

        unique = []
        for s in sols:
            s = to_radians_if_needed(s)
            if s is None:
                continue
            Tchk = fk_puma(s, l1, l2, l3)
            if not same_pose(Tref, Tchk, tol_pos, tol_ang):
                continue
            # отфильтровываем дубликаты по позе
            if any(same_pose(Tchk, fk_puma(u, l1, l2, l3), tol_pos, tol_ang) for u in unique):
                continue
            unique.append(s)
            _, origins = fk_with_origins(s, l1, l2, l3)
            path = f"images/example_{i}_config_{len(unique)}.png"
            title = f"Цель {i}, конфигурация {len(unique)}\nθ[рад]={np.round(s,3)}"
            plot_arm(origins, title, path)
            saved.append(path)
            if len(unique) >= max_per_target:
                break

    return saved


# запуски
sample_workspace(n=2000, seed=42)
files = visualize_examples(num_targets=3, seed=7)
print("Сохранено:", files)


Сохранено: ['images/example_1_config_ref.png', 'images/example_2_config_ref.png', 'images/example_3_config_ref.png']


<!-- строка 1: три 3D-конфигурации -->
| ![cfg1](images/example_1_config_ref.png) | ![cfg2](images/example_2_config_ref.png) | ![cfg3](images/example_3_config_ref.png) |
|:--:|:--:|:--:|
| Конфигурация 1 | Конфигурация 2 | Конфигурация 3 |

<!-- строка 2: матрица охвата и ошибка -->
| ![coverage](images/ik_pos_error_hist.png) | ![error](images/workspace_points.png) |
|:--:|:--:|
| Матрица охвата | Карта ошибки |
